In [ ]:
%pylab inline

In [ ]:
import scipy
import scipy.ndimage

In [ ]:
from opensimplex import OpenSimplex

In [ ]:
# Generates a cultive field, pass in a "base field" and it will assign a new type of cultive
# to the areas where the noise_filter parameter passes the test
# The category represents the "id" of the cultive

def gen_cultive(field, noise_generator, noise_freq, noise_filter, category):
    for x in range(1,100):
        for y in range(1,100):
            nv = noise_generator.noise2d(0.06 * x, 0.06 * y)
            if nv > noise_filter:
                field[x,y] = category
    return field

In [ ]:
def makeGaussian(size, fwhm = (3., 3.), center=None):
    """ Make a square gaussian kernel.

    size is the length of a side of the square
    fwhm is full-width-half-maximum, which
    can be thought of as an effective radius.
    """

    x = np.arange(0, size, 1, float)
    y = x[:,np.newaxis]

    if center is None:
        x0 = y0 = int(size / 2.)
    else:
        x0 = center[0]
        y0 = center[1]

    return np.exp(-4.*np.log(2) * ( ((x-x0)**2 / fwhm[0]) + ((y-y0)**2 / fwhm[1])  ) )

In [ ]:
seed = numpy.random.randint(1000)
tmp = OpenSimplex(seed=seed)
seed2 = numpy.random.randint(1000)
tmp2 = OpenSimplex(seed=seed2)

In [ ]:
# Generate a cultive map
# This is the base cultive, id=1
wfield = np.full( (100, 100), 1)

# Then another cultive with id=2
gen_cultive(wfield, tmp, 0.004, 0.05, 2);

# And a final with id=3
gen_cultive(wfield, tmp2, 0.0045, 0.25, 3);


In [ ]:
# Here you can study the frequencies of each type of cultive
plt.hist(wfield);

In [ ]:
# Random cultive representation map
plt.imshow(wfield)

In [ ]:
# Here we generate the infectivity distribution

# makeGaussian(...) retruns a gaussian of "infected_range" size and \theta_x, \theta_y are the distribution means

# After that we can rotate the gaussian to incorporate the direction of the "wind".
# theta_x and theta_y have to change according to the x and y magnitude vectors of the wind

infection_range = 19
theta_x = 95.
theta_y = 30.
wind_angle = 60.
plt.imshow(scipy.ndimage.interpolation.rotate(makeGaussian(infection_range, fwhm=(theta_x, theta_y)), wind_angle))

In [ ]:
plt.imshow(scipy.ndimage.interpolation.rotate(makeGaussian(5, fwhm=(20., 9.)), 70))

In [ ]:
# Start writing the infection model

infected_field = numpy.zeros( (100,100) )

# New crop infection rate
infection_rate = 0.002

# New infection site probability
infection_site_rate = 0.08

# Array of (x,y) position of initial infection sites, gaussian distributed infection probabilities centered at (x,y)
infection_sites = []

# Infection range is the size of the gaussian
infection_range = 5

# Wind direction vector (x,y) the magnitude can be extracted from the coordinates
# The wind can change over time and will be used to modify rotate, scale and/or recenter the gaussian distributions
wind = (0,0)


In [ ]:
# You can rerun this step to look at how the pathogen spreads over time
for t in range(1,1000):
    p = numpy.random.rand()
    # The parameters for the gaussian function are to be modified by the wind direction and strength
    gaussian = scipy.ndimage.interpolation.rotate(makeGaussian(infection_range, fwhm=(20., 9.)), 70)
    
    
    if p<infection_site_rate:
        # Place a new random infection site in the field
        infection_sites.append( (numpy.random.randint(wfield.shape[0]), numpy.random.randint(wfield.shape[1])) )
    
    # For each infection site, increase the "infectedness" of the pixel
    for infection_site in infection_sites:
        for isx in range(-int(infection_range/2), int(infection_range/2)):
            # The modulus is to stay within bounds
            x = (infection_site[0]+isx) % infected_field.shape[0]
            for isy in range(-int(infection_range/2), int(infection_range/2)):
                y = (infection_site[1]+isy) % infected_field.shape[1]
                infection_prob = gaussian[isx,isy] * 1./wfield[x,y] # Prob * 1./category
                infected_field[x][y] += 1 if infection_prob<infection_rate else 0

# Clear the infection_sites before doing a new timestep
infection_sites=[]
plt.imshow(infected_field, alpha=0.9)
plt.imshow(wfield, alpha=0.5)

In [ ]:
len(infection_sites)